In [ ]:
#This is code to compare individual raw trajectories with the avergae polynomial fit 
#of all trajectories in a folder 

#IN: .csv files from labelling trajectory in imageJ (note y inversion incorporated into code) 

#OUT: List of SSE for each trial 


#CALCULATIONS
#1. Calculate polynomial fit for each trial in folder 
#2. Get average polynomial equation using from all of the trials 
#3. Go back and use random drop out to normalize # of coordinates for each raw trajectory 
#4. For each x value in raw data, solve average polynomial equation for y 
#5. Calculate sse using difference between average y and raw y 


#PLEASE NOTE: My video data has perspective shift due to angled view 
#e.g. coordinates vary based on where mouse is located in z axis 
#x,y shift is compensated for in this code but can be hashed out


In [ ]:
##DEPENDENCIES##

import pandas as pd 
import os, sys
import matplotlib.pyplot as plt 
from numpy.polynomial import Polynomial
import numpy as np
from random import shuffle 

In [ ]:
#DIRECTORY AND VARIABLES 

directory = ''#directory to files

 
degree = 3 #degree for polynomial fitting 

#range is used for polyfit/graphing 
range1 = 320 #x value ~minimum 
range2 = 380 #x value ~maximum 

#global xmin and ymin to compensate for perspective shift
#z shift is not compensated for (~8 pixels max)
#hash out if not needed
xmin = 311
ymin = 160

#which trajectory has lowest frame #? use for dropout normalization 
lowestframe = 20 

In [ ]:
#create df shell 
Ydf = pd.DataFrame({'index' : list(range(range1,range2))})
Ydf.set_index('index')

#fit a polynomial to each trajectory in directory 
i_file = 0;
for filename in sorted(os.listdir(directory)):
    if filename.endswith(".csv"):
        i_file = i_file + 1;
        name = os.path.join(directory, filename)
        df = pd.read_csv(name)
        
        #Compensation for x,y perspective shift
        xdiff = df['X'][0] - xmin
        ydiff = df['Y'][0] - ymin
        df['newX'] = df.X.apply(lambda x: x - xdiff)
        df['newY'] = df.Y.apply(lambda x: x - ydiff)
        
        #invert y values due to imageJ labelling convention 
        df['newY'] = df.newY.apply(lambda x: max(df['newY']) - x)
        
        #extract values from df
        x = df['newX'].values
        y = df['newY'].values
        
        #plot raw if you want to look at it
        #plt.plot(x,y, color = 'k', alpha = 0.2)

        #polynomial fit 
        z = np.polyfit(x, y, degree)
        fx = np.poly1d(z)
        
        #solve for y of polynomial equation at predetermined range
        ranges = list(range(range1,range2))
        num = [] 
        for i in ranges: 
            #parse out extreme outliers (e.g. going to -∞ if trajectory is short)
            if (np.polyval(z,i)) > (-5):
                num.append(np.polyval(z, i))
            else: 
                num.append(None)
                
        #each column is the values of the fitted polynomial     
        Ydf[str(i_file)] = num

#get average of all the polynomial trajectories in df, ignoring nan
Ydf = Ydf.set_index('index')
Ymean = Ydf.mean(axis = 1)
Ymean = Ymean.reset_index()
Ymean.columns = ['x','y']

#extract values of x and y from averaged polynomial 
mean_x = Ymean['x'].values
mean_y = Ymean['y'].values

#get an equation from mean_x and mean_y values
mean_z = np.polyfit(mean_x, mean_y, degree)
mean_fx = np.poly1d(mean_z)
        

In [ ]:
#list of sse to be filled 
sse = [] 

#iterating through files in directory 
i_file = 0;
for filename in sorted(os.listdir(directory)):
    if filename.endswith(".csv"):
        i_file = i_file + 1;
        name = os.path.join(directory, filename)
        df = pd.read_csv(name)
        
        print("...analyzing" + str(filename))
        print('')
        
        #Compensation for x,y perspective shift
        xdiff = df['X'][0] - xmin
        ydiff = df['Y'][0] - ymin
        df['newX'] = df.X.apply(lambda x: x - xdiff)
        df['newY'] = df.Y.apply(lambda x: x - ydiff)
        
        #invert y values due to imageJ labelling convention 
        df['newY'] = df.newY.apply(lambda x: max(df['newY']) - x)
        
        #change x vals to int for later 
        df['newX'] = df.newX.astype(int)
        
        #drop duplicates 
        df.drop_duplicates(subset = ['newX'], inplace = True)
        
        #extract values and do random dropout so each trial has 
        #same number of data points for sse calculations
        x = df['newX'].values
        xshuffle = x.tolist()
        xshuffle = [int(x) for x in xshuffle]
        shuffle(xshuffle)
        x2graph = sorted(xshuffle[0:lowestframe])
        
        #find raw y value @ each raw x value from the trajectory
        y_raw = [] 
        for x in x2graph: 
            string2input = ('newX == ' + str(x))
            y_raw.append((df.query(string2input)['newY']))
        y_raw = [x.values for x in y_raw]
        y_raw = np.asarray(y_raw)
        y_raw = y_raw.flatten()

        
        #find average y value @ each raw x value from the trajectory 
        ymean_fitted = [] 
        for x in x2graph: 
            if (np.polyval(mean_z, x)) > -5: #prevent skewing due to polynomial going to ∞
                ymean_fitted.append(np.polyval(mean_z, x))
            else: 
                ymean_fitted.append(0)
        ymean_fitted = np.asarray(ymean_fitted)
        ymean_fitted = ymean_fitted.flatten() 
        
        #sse 
        sse.append(np.sum((ymean_fitted - y_raw)**2))

        
        